# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'city_weather.csv')

city_df = pd.read_csv(csv_file)
city_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Yellowknife,20,CA,1616149752,85,62.4560,-114.3525,12.20,9.22
1,Aplahoué,100,BJ,1616150073,47,6.9333,1.6833,91.71,5.37
2,Okhotsk,0,RU,1616149948,68,59.3833,143.3000,7.81,13.60
3,Klaksvík,73,FO,1616149762,93,62.2266,-6.5890,46.40,9.22
4,São Félix do Xingu,98,BR,1616150040,97,-6.6447,-51.9950,71.76,2.62
...,...,...,...,...,...,...,...,...,...
542,Lalibela,0,ET,1616150223,16,12.0333,39.0333,80.24,3.69
543,Churapcha,100,RU,1616150223,95,62.0025,132.4325,10.49,7.29
544,Black River,20,JM,1616150223,78,18.0264,-77.8487,73.40,3.44
545,Guerrero Negro,35,MX,1616149755,85,27.9769,-114.0611,55.92,9.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key=gkey)

In [19]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#CRITERIA: A max temperature lower than 80 degrees but higher than 70. 
# Wind speed less than 10 mph.
# Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

narrowed_city_df = city_df.loc[(city_df["Max Temp"] > 70) & (city_df["Max Temp"] < 80) & (city_df["Cloudiness"] == 0), :]
#Drop any rows will null values.
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05
1,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75
2,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22
3,Esperance,0,AU,1616149760,53,-33.8667,121.9000,75.20,13.80
4,Arraial do Cabo,0,BR,1616149748,94,-22.9661,-42.0278,75.20,4.61


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
hotel_df=narrowed_city_df
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05,
1,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75,
2,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22,
3,Esperance,0,AU,1616149760,53,-33.8667,121.9000,75.20,13.80,
4,Arraial do Cabo,0,BR,1616149748,94,-22.9661,-42.0278,75.20,4.61,
5,Nouakchott,0,MR,1616150083,20,18.0858,-15.9785,77.00,17.27,
6,Saldanha,0,ZA,1616149806,68,-33.0117,17.9442,71.60,17.27,
7,São João da Barra,0,BR,1616149789,88,-21.6403,-41.0511,75.20,4.61,


In [22]:

for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": gkey,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Puerto Escondido,0,MX,1616149973,65,15.8500,-97.0667,74.48,4.05,Selina Puerto Escondido
1,Sakakah,0,SA,1616150084,23,29.9697,40.2064,71.60,5.75,Raoum Inn Hotel
2,Rock Sound,0,BS,1616149778,82,24.9000,-76.2000,73.72,16.22,Four Points Suite
3,Esperance,0,AU,1616149760,53,-33.8667,121.9000,75.20,13.80,Esperance Chalet Village
4,Arraial do Cabo,0,BR,1616149748,94,-22.9661,-42.0278,75.20,4.61,Orlanova Boutique Hotel
5,Nouakchott,0,MR,1616150083,20,18.0858,-15.9785,77.00,17.27,AZALAÏ HOTEL NOUAKCHOTT (MARHABA )
6,Saldanha,0,ZA,1616149806,68,-33.0117,17.9442,71.60,17.27,Saldanha Bay Hotel
7,São João da Barra,0,BR,1616149789,88,-21.6403,-41.0511,75.20,4.61,Pousada Kactus


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig



Figure(layout=FigureLayout(height='420px'))